In [1]:
import spacy
from itertools import chain, groupby
from operator import itemgetter
from sklearn.cluster import DBSCAN
import gensim
import numpy as np

from solrhandler import SolrHandler

In [2]:
def preprocess_text(text, nlp_model):
    doc = nlp_model(text)
    text = " ".join([tok.lemma_ for tok in doc if not tok.is_stop and tok.is_alpha])
    return text.lower()

In [3]:
nlp = spacy.load("de_core_news_lg")

In [4]:
#path2model = "/Users/jakob/Downloads/german.model"
#model = gensim.models.KeyedVectors.load_word2vec_format(path2model, binary=True)

In [5]:
handler = SolrHandler(max_elems=20000)

In [6]:
df = handler.get_df_from_query("*%3A*")

In [7]:
lemma_lst = [" ".join(lst) for lst in df.ssdsLemma.tolist()]

In [8]:
lemma_lst = [preprocess_text(lemma_str, nlp) for lemma_str in lemma_lst]

---

In [9]:
def has_vec(word, model):
    try:
        model.get_vector(word)
    except:
        return False
    return True

In [10]:
lemmas_str = " ".join(lemma_lst)
lemmas = set([tok.text for tok in nlp(lemmas_str)])

In [11]:
lemmas_vectorizable = [lemma for lemma in lemmas if nlp(lemma)[0].has_vector]
lemmas_not_vectorizable = [lemma for lemma in lemmas if lemma not in lemmas_vectorizable]
len(lemmas_vectorizable), len(lemmas_not_vectorizable)

(1994, 3942)

In [12]:
lemmas_vectorizable_vecs = [nlp(lemma)[0].vector for lemma in lemmas_vectorizable]
#lemmas_vectorizable_vecs.shape

In [13]:
clustering = DBSCAN(eps=.15, min_samples=1, metric="cosine")
labels = clustering.fit_predict(lemmas_vectorizable_vecs)
len(lemmas_vectorizable), len(set(labels))

(1994, 1881)

In [14]:
elems = [(tok, label) for tok, label in zip(lemmas_vectorizable, labels)]
clusters = [[el for el in elems if el[1] == i] for i in range(len(set(labels)))]
[c for c in clusters if len(c) > 1]

[[('schädigung', 2), ('entschädigung', 2), ('beschädigung', 2)],
 [('bildung', 8), ('fortbildung', 8), ('weiterbildung', 8)],
 [('planungsrechtlich', 15),
  ('zivilrechtlich', 15),
  ('strafrechtlich', 15),
  ('verkehrsrechtlich', 15),
  ('völkerrechtlich', 15),
  ('außergerichtlich', 15),
  ('juristisch', 15),
  ('planungsrechtlichen', 15),
  ('zivilrechtliche', 15),
  ('rechtlich', 15),
  ('gerichtlich', 15)],
 [('beratung', 21), ('rechtsberatung', 21)],
 [('gutachten', 37), ('gutachter', 37)],
 [('sicherheitstechnik', 53),
  ('sicherheitsrelevant', 53),
  ('sicherheitstechnisch', 53)],
 [('neuwagen', 55), ('gebrauchtwagen', 55)],
 [('nebenwirkung', 58), ('wirkung', 58)],
 [('arbeitnehmer', 60), ('arbeitgeber', 60)],
 [('seelisch', 89), ('psychisch', 89)],
 [('fachschule', 99),
  ('hochschule', 99),
  ('fachhochschule', 99),
  ('berufsschule', 99)],
 [('änderung', 102), ('veränderung', 102)],
 [('erklärung', 123), ('aufklärung', 123), ('klärung', 123)],
 [('benötigen', 166), ('brauch